In [1]:
import pymysql
con= pymysql.connect(
    host="localhost",
    user="root",
    password="123456789",
    autocommit=True
    )
print(con)

In [2]:
# Read the CSV files into pandas dataframes

import pandas as pd

df1 = pd.read_csv('customers.csv')
df2 = pd.read_csv('restaurants.csv')
df3 = pd.read_csv('orders.csv')
df4 = pd.read_csv('deliveries.csv')

In [3]:
mycursor = con.cursor()

In [4]:
print(df1.columns)

Index(['customer_id', 'name', 'email', 'phone', 'address', 'signup_date',
       'is_premium', 'preferred_cuisine', 'total_orders', 'average_rating'],
      dtype='object')


In [5]:
print(len(df1.columns))

10


In [6]:
print(df2.columns)

Index(['restaurant_id', 'name', 'cuisine_type', 'location', 'owner_name',
       'average_delivery_time', 'contact_number', 'rating', 'total_orders',
       'is_active'],
      dtype='object')


In [7]:
print(len(df2.columns))

10


In [8]:
mycursor.execute("USE ZOMATO_DB")

0

In [9]:
insert_query = """
    INSERT INTO customers (customer_id, name, email, phone, address, signup_date, is_premium, preferred_cuisine, total_orders, average_rating)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
for index, row in df1.iterrows():
    mycursor.execute(insert_query,(
        row['customer_id'],
        row['name'],
        row['email'],
        row['phone'],
        row['address'],
        row['signup_date'],
        row['is_premium'],
        row['preferred_cuisine'],
        row['total_orders'],
        row['average_rating'],
    
        ))

IntegrityError: (1062, "Duplicate entry '1' for key 'customers.PRIMARY'")

In [ ]:
insert_query = """
    INSERT INTO Restaurants (restaurant_id, name, cuisine_type, location, owner_name, average_delivery_time, contact_number, rating, total_orders, is_active)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
for index, row in df2.iterrows():
    mycursor.execute(insert_query,(
        row['restaurant_id'],
        row['name'],
        row['cuisine_type'],
        row['location'],
        row['owner_name'],
        row['average_delivery_time'],
        row['contact_number'],
        row['rating'],
        row['total_orders'],
        row['is_active'],
    
        ))

In [ ]:
print(df3.columns)

In [ ]:
print(df3['order_id'].duplicated().sum()) 

print(df3[df3['order_id'].duplicated(keep=False)]) 

In [40]:
df3 = df3.drop_duplicates(subset='order_id')

In [ ]:
print(df3.head())

In [15]:
insert_query = """
    INSERT INTO orders (order_id, customer_id, restaurant_id, items, quantity, order_date, delivery_time, order_statuses, total_amount, payment_mode, discount_applied, feedback_rating)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
for index, row in df3.iterrows():
    mycursor.execute(insert_query,(
        row['order_id'],
        row['customer_id'],
        row['restaurant_id'],
        row['items'],
        row['quantity'],
        row['order_date'],
        row['delivery_time'],
        row['order_statuses'],
        row['total_amount'],
        row['payment_mode'],
        row['discount_applied'],
        row['feedback_rating'],

        ))
    


In [ ]:
print(len(df4.columns))

In [ ]:
insert_query = """
    INSERT INTO deliveries (delivery_id, order_id, delivery_person_id, delivery_status, distance, delivery_time, estimated_time, delivery_fee, vehicle_type)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
for index, row in df4.iterrows():
    mycursor.execute(insert_query,(
        row['delivery_id'],
        row['order_id'],
        row['delivery_person_id'],
        row['delivery_status'],
        row['distance'],
        row['delivery_time'],
        row['estimated_time'],
        row['delivery_fee'],
        row['vehicle_type']

        ))
    
# Commit changes and close connection
con.commit()
mycursor.close()
con.close()
print("Data inserted successfully")

In [10]:
import pymysql

In [11]:
class DatabaseOperator:
    def __init__(self, host="localhost", user="root", password="123456789", db_name="zomato_db"):
        self.host = host
        self.user = user
        self.password = password
        self.db_name = db_name
        

    def connect(self):
        return pymysql.connect(
            host=self.host,
            user=self.user,
            password=self.password,
            database=self.db_name,
            cursorclass=pymysql.cursors.DictCursor
        )
        
    def execute_query(self, query, params=None):
        """
        Execute any SQL query with error handling and return the result.

        Args:
            query (str): The SQL query to execute.
            params (tuple, optional): Parameters for parameterized queries.

        Returns:
            list: Query result as a list of dictionaries for SELECT queries.
            bool: True if INSERT/UPDATE/DELETE is successful, False if an error occurs.
        """
        connection = self.connect()
        cursor = connection.cursor(pymysql.cursors.DictCursor)
        
        try:
            # Execute with or without parameters
            if params:
                cursor.execute(query, params)
            else:
                cursor.execute(query)

            # For SELECT queries, return fetched data
            if query.strip().lower().startswith("select"):
                result = cursor.fetchall()
                return result
            else:
                # For INSERT, UPDATE, DELETE, commit the transaction
                connection.commit()
                return True

        except pymysql.MySQLError as sql_err:
            print(f"SQL Error: {sql_err}")
            return False

        except Exception as e:
            print(f"Unexpected Error: {e}")
            return False

        finally:
            cursor.close()
            connection.close()
    

    def create_customer(self, name, email, phone, address):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute("""
                    INSERT INTO customers (name, email, phone, address) 
                    VALUES (%s, %s, %s, %s)
                """, (name, email, phone, address))
                conn.commit()

    def read_customers(self):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute("SELECT * FROM customers")
                return mycursor.fetchall()

    def update_customer(self, customer_id, name, email):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                if name:
                    mycursor.execute("UPDATE customers SET name=%s WHERE customer_id=%s", (name, customer_id))
                if email:
                    mycursor.execute("UPDATE customers SET email=%s WHERE customer_id=%s", (email, customer_id))
                conn.commit()

    def delete_customer(self, customer_id):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute("DELETE FROM customers WHERE customer_id=%s", (customer_id,))
                conn.commit()

    def alter_customer_table(self, new_column, data_type):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute(f"ALTER TABLE customers ADD COLUMN {new_column} {data_type}")
                conn.commit()

# Restaurants Table

    def create_restaurant(self, name, location):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute("INSERT INTO restaurants (name, location) VALUES (%s, %s)", (name, location))
                conn.commit()

    def read_restaurants(self):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute("SELECT * FROM restaurants")
                return mycursor.fetchall()

    def update_restaurant(self, restaurant_id, name, location):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                if name:
                    mycursor.execute("UPDATE restaurants SET name=%s WHERE restaurant_id=%s", (name, restaurant_id))
                if location:
                    mycursor.execute("UPDATE restaurants SET location=%s WHERE restaurant_id=%s", (location, restaurant_id))
                conn.commit()

    def delete_restaurant(self, restaurant_id):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute("DELETE FROM restaurants WHERE restaurant_id=%s", (restaurant_id,))
                conn.commit()

    def alter_restaurant_table(self, new_column, data_type):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute(f"ALTER TABLE restaurants ADD COLUMN {new_column} {data_type}")
                conn.commit()
                
# Orders Table 

    def create_order(self, customer_id, restaurant_id, items, quantity):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute("""
                    INSERT INTO orders (customer_id, restaurant_id, items, quantity) 
                    VALUES (%s, %s, %s, %s)
                """, (customer_id, restaurant_id, items, quantity))
                conn.commit()

    def read_orders(self):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute("SELECT * FROM orders")
                return mycursor.fetchall()

    def update_order(self, order_id, items, quantity):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                if items:
                    mycursor.execute("UPDATE orders SET item=%s WHERE order_id=%s", (items, order_id))
                if quantity:
                    mycursor.execute("UPDATE orders SET quantity=%s WHERE order_id=%s", (quantity, order_id))
                conn.commit()

    def delete_order(self, order_id):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute("DELETE FROM orders WHERE order_id=%s", (order_id,))
                conn.commit()

    def alter_order_table(self, new_column, data_type):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute(f"ALTER TABLE orders ADD COLUMN {new_column} {data_type}")
                conn.commit()
                
# Deliveries Table 

    def create_delivery(self, order_id, delivery_status):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute("""
                    INSERT INTO deliveries (order_id, delivery_status) 
                    VALUES (%s, %s)
                """, (order_id, delivery_status))
                conn.commit()

    def read_deliveries(self):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute("SELECT * FROM deliveries")
                return mycursor.fetchall()

    def update_delivery(self, delivery_id, delivery_status):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute("UPDATE deliveries SET delivery_status=%s WHERE delivery_id=%s", (delivery_status, delivery_id))
                conn.commit()

    def delete_delivery(self, delivery_id):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute("DELETE FROM deliveries WHERE delivery_id=%s", (delivery_id,))
                conn.commit()

    def alter_delivery_table(self, new_column, data_type):
        with self.connect() as conn:
            with conn.cursor() as mycursor:
                mycursor.execute(f"ALTER TABLE deliveries ADD COLUMN {new_column} {data_type}")
                conn.commit()


In [12]:
db = DatabaseOperator(user="root", password="123456789", db_name="zomato_db")

In [13]:
# Add a new customer

db.create_customer("Ram", "alice@example.com", "1234567890", "123, Main Street")

In [14]:
# Read all customers

print(db.read_customers())

[{'customer_id': 1, 'name': 'Alice Wonderland', 'phone': '260.331.0778', 'email': 'alice@gmail.com', 'address': '35647 Smith Corner Suite 569\nSouth Stephanieville, MD 07703', 'signup_date': datetime.date(2021, 6, 8), 'is_premium': 1, 'preferred_cuisine': 'Chinese', 'total_orders': 65, 'average_rating': Decimal('3.1'), 'loyalty_points': 0, 'Recipee': 0, 'Sweet': None, 'Amount': None}, {'customer_id': 2, 'name': 'Shannon Sanders', 'phone': '960-507-5941', 'email': 'sharon09@example.com', 'address': '637 Finley Cliffs\nSouth Emily, NM 19536', 'signup_date': datetime.date(2020, 7, 25), 'is_premium': 0, 'preferred_cuisine': 'Chinese', 'total_orders': 60, 'average_rating': Decimal('1.7'), 'loyalty_points': 0, 'Recipee': 0, 'Sweet': None, 'Amount': None}, {'customer_id': 5, 'name': 'Barry Brady', 'phone': '(885)619-3227x3520', 'email': 'melissawilson@example.com', 'address': '1304 Juan Crescent Suite 716\nElizabethport, CA 49436', 'signup_date': datetime.date(2020, 4, 24), 'is_premium': 1, '

In [15]:
# Update a customer's name

db.update_customer(1, name="Alice Wonderland", email="alice@gmail.com")

In [10]:
# Delete a customer

db.delete_customer(3)

In [ ]:
# Add a new column to customers table

db.alter_customer_table("Recipee", "INT DEFAULT 0")